> 27183 food items with reviews

In [1]:
from datetime import datetime
startTime = datetime.now()

** retrive Burpple data from ES **

In [3]:
%load_ext autotime
%load_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'20180410'

time: 361 ms


In [4]:
# burpple parameters
burpples = f.burpple_para()

time: 14.5 ms


In [5]:
i = 1
dfs = []

time: 16.8 ms


In [8]:
# retrive data - part 1
if 1==1:
    recs = f.retrive_data(burpples[i], "review")

time: 52.1 s


In [9]:
df = f.select_burpple_fields(recs)
dfs.append(df)
i = i-1

time: 54.3 s


In [10]:
# retrive data - part 0
if 1==1:
    recs = f.retrive_data(burpples[i], "review")

time: 7.14 s


In [11]:
df = f.select_burpple_fields(recs)
dfs.append(df)

time: 4.65 s


** consolidate burpple records **

In [12]:
df = f.consolidate_burpple_records(dfs)

time: 6.84 s


In [13]:
print df.shape
print df.columns.tolist()
df.sample(2, random_state=2008)

(323104, 9)
['_id', 'loc', '_source.title', '_source.body', '_source.crawlTimeStamp', '_source.foodImgUrl', '_source.username', '_source.restaurant.id', '_source.restaurant.name']


,_id,loc,_source.title,_source.body,_source.crawlTimeStamp,_source.foodImgUrl,_source.username,_source.restaurant.id,_source.restaurant.name
242662,jnF_zvaY,burppleinitial/review/jnF_zvaY,"Aged Cheddar, Fig & #Cognac Jam and Candied Wa...","Aged Cheddar, Fig & #Cognac Jam and Candied Wa...",2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/669ed7d8855d40aab...,@18930,79509,The Humble Loaf
74846,CzDetUln,burppleinitial/review/CzDetUln,Orange Brioche,,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/18d18ccb332f616d2...,@iSteven,116616,Artisan Boulangerie Co. (Great World City)


time: 222 ms


** obtain review objects with clean text **

In [15]:
# add new columns
import pandas as pd
pd.options.mode.chained_assignment = None # default is warn
# cols = ['loc','_source.title','_source.body','_source.restaurant.name']
# df = df[cols] 
df.columns = [col.replace("_source.", "") for col in df.columns.tolist()]
# decompose full review into respetive cols
df["full_review"] = df['title'] + " " + df['body']
# twitter style cleaning
df["review_text"], df["hashtags"], df["mentions"], df["dollars"], df["scores"], df["urls"] = \
zip(*df["full_review"].apply(f.clean_review))
df = df[[ col for col in df.columns.tolist() if col != "full_review"]]
# On review text, perform clean_name_v2()
df["clean_review"] = df["review_text"].apply(f.clean_name_v2)

time: 59 s


In [16]:
print df.shape
print df.columns.tolist()
df.sample(2,random_state=2008)

(323104, 16)
['_id', 'loc', 'title', 'body', 'crawlTimeStamp', 'foodImgUrl', 'username', 'restaurant.id', 'restaurant.name', 'review_text', 'hashtags', 'mentions', 'dollars', 'scores', 'urls', 'clean_review']


,_id,loc,title,body,crawlTimeStamp,foodImgUrl,username,restaurant.id,restaurant.name,review_text,hashtags,mentions,dollars,scores,urls,clean_review
242662,jnF_zvaY,burppleinitial/review/jnF_zvaY,"Aged Cheddar, Fig & #Cognac Jam and Candied Wa...","Aged Cheddar, Fig & #Cognac Jam and Candied Wa...",2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/669ed7d8855d40aab...,@18930,79509,The Humble Loaf,"Aged Cheddar, Fig and Jam and Candied Walnuts ...","(#Cognac, #lunch, #weekend, #Cognac, #lunch, #...",(),(),(),(),aged cheddar fig and jam and candied walnuts o...
74846,CzDetUln,burppleinitial/review/CzDetUln,Orange Brioche,,2017-04-03 11:00:52.282519,https://s3.burpple.com/foods/18d18ccb332f616d2...,@iSteven,116616,Artisan Boulangerie Co. (Great World City),Orange Brioche,(),(),(),(),(),orange brioche


time: 1.13 s


** retrive food entities **

In [17]:
file_name = "food_entities__20180410.p"
food = f.retrive_file(file_name)

retrived: food_entities__20180410.p
time: 36.7 s


In [18]:
food_entities = food.reset_index()
# remove food of " and "-PRON-"
food_entities = food_entities[~food_entities["assigned_name"].isin(["", "-PRON-"])]
# food name variant : food_item_index
index_ref = dict()
def add_ref(line):
    v = line["index"]
    index_ref.update({k:v for k in line["name_variants"]})
food_entities.apply(add_ref, axis =1)
food_items = index_ref.keys()
print "number of food searched for: %d"%len(food_items)

number of food searched for: 143255
time: 4.3 s


In [19]:
file_name = "words_to_remove_20180308.csv"
rmv = pd.read_csv(file_name, header=None)[0].tolist()
rmv = rmv+['can', 'sweet', 'love', 'up', 'no', 'me', 'back', 'fluffy', 'hot', 'spicy', 'crisp', 'salty', 'house', 'red', 'fat', 'roasted', 'white', 'refreshing', 'black', 'cold', 'deep fried', 'pieces', 'toasted', 'skin', 'cheesy', 'hungry', 'pop', 'miss', 'option', 'fried', 'tomato', 'vanilla', 'plus', 'potato', 'thai', 'spice', 'infused', 'sauces', 'to share', 'lover', 'surprise', 'balanced', 'choice', 'french', 'green', 'steamed', 'italian', 'wake up', 'silver', 'savoury flavours', 'western', 'set for', 'pan fried', 'the original', 'decadence', 'the great', 'uplifting', 'all in', 'meaty', 'spice level', 'per slice', 'discovery', 'smokey', 'all time favourites', 'sticky', 'the double', 'pint', 'mad', 'singapore style', 'alcoholic', 'serves', 'nutty', 'salted', 'lunch set', 'boost', 'anything', 'dips', 'vietnamese', 'set lunch', 'lager', 'for pax', 'below', 'bundle']

time: 23.9 ms


In [20]:
print "number of food items removed: %d"%len(rmv)

number of food items removed: 434
time: 14.9 ms


** search for food entities **

In [22]:
"""
saved: burpple__review__food__20180320.p
CPU times: user 5h 33min 18s, sys: 2min 48s, total: 5h 36min 6s
Wall time: 5h 33min 4s
"""
def search_food(review):
    return [s for s in f.search_food(review, values=food_items) if s not in rmv]

if 1==1:
    df["food_items"] = df["clean_review"].apply(search_food)    
    # ref name_variation to food index
    df["food_index"] = df["food_items"].apply(lambda l: 
                                              sorted([index_ref[s] for s in l]))
    # remove original review_text
    cols = df.columns.tolist()
    cols.remove("review_text")
    df = df[cols]

CPU times: user 5h 33min 18s, sys: 2min 48s, total: 5h 36min 6s
Wall time: 5h 33min 4s


In [23]:
file_name = "burpple__review__food__"+f.today()+".p" 
f.save_file(df, file_name)

saved: burpple__review__food__20180320.p


In [33]:
## save to json
file_name = "__".join(["burpple_reviews", f.today()]) + ".json"
df.set_index("loc").to_json(file_name, orient='index')
print "saved: %s"%file_name

saved: burpple_reviews__20180321.json


In [24]:
df.sample(2, random_state=2008)

,loc,title,body,restaurant.name,hashtags,mentions,dollars,scores,urls,clean_review,food_items,food_index
242662,burppleinitial/review/jnF_zvaY,"Aged Cheddar, Fig & #Cognac Jam and Candied Wa...","Aged Cheddar, Fig & #Cognac Jam and Candied Wa...",The Humble Loaf,"(#Cognac, #lunch, #weekend, #Cognac, #lunch, #...",(),(),(),(),aged cheddar fig and jam and candied walnuts o...,"[toast, jam, walnuts]","[food_item_102154, food_item_47881, food_item_..."
74846,burppleinitial/review/CzDetUln,Orange Brioche,,Artisan Boulangerie Co. (Great World City),(),(),(),(),(),orange brioche,[orange brioche],[food_item_65293]


In [25]:
freq = f.count_freq(f.flatten(df["food_items"].tolist()))
print "number of food name variations itendtified: %d"%len(freq)

number of food name variations itendtified: 27183


In [26]:
freq = f.count_freq(f.flatten(df["food_index"].tolist()))
print "number of unique food items itendtified: %d"%len(freq)

number of unique food items itendtified: 21273


In [27]:
# most popular food items
freq[:10]

[('food_item_22793', 10169),
 ('food_item_57771', 8579),
 ('food_item_17273', 8081),
 ('food_item_46138', 7774),
 ('food_item_13603', 7614),
 ('food_item_101898', 7377),
 ('food_item_29415', 7280),
 ('food_item_76059', 6474),
 ('food_item_86592', 5984),
 ('food_item_63740', 5882)]

** link review locs to entities ** 

In [28]:
# obtain review locs for each food item
loc_ref = df.set_index("loc").to_dict()["food_index"]
inv_loc_ref = f.inverse_dict(loc_ref)
# append review locs to food entities
food.reset_index(inplace=True)
def map_loc(s):
    try:
        return inv_loc_ref[s]
    except:
        return []
food["review_loc"] = food["index"].apply(map_loc)
food.set_index("index", inplace=True)

In [29]:
food.sample(2, random_state=2008)

,assigned_name,name_variants,menu_name,tag,price,desc,vendors,identification_index,loc,timestamp,price_average,review_loc
index,,,,,,,,,,,,
food_item_13994,cannelloni di malanzane,[cannelloni di malanzane],"[(Cannelloni Di Malanzane, 15)]",[Specialita],"[31.03, 31.03, 31.03, 31.03, 31.03, 31.03, 31....","[, Baked eggplant cannelloni, goat ricotta che...","[delivery_3055, delivery_3056]","(food_17362,)","[deliveroo/food/39541, deliveroo/food/82634, d...","[2017-07-03T17:55:01.668428, 2017-07-13T14:37:...",31.03,[]
food_item_6491,beef mandi,[beef mandi],"[(Beef Mandi, 18)]","[Favourites, Mandi]","[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[, Default]",[delivery_199],"(food_8022,)","[foodpanda/menu_item/112620, foodpanda/menu_it...","[2017-07-11 01:13:18.670689, 2017-07-15 00:25:...",8.00,[]


In [36]:
## save to json
file_name = "__".join(["food_entities", f.today()]) + ".json"
food.to_json(file_name, orient='index')
print "saved: %s"%file_name

saved: food_entities__20180321.json


In [37]:
## save to p file
file_name = "__".join(["food_entities", f.today()]) + ".p"
f.save_file(food, file_name)

saved: food_entities__20180321.p
